In [2]:
from neuron import h
from neuron.units import ms, mV
%matplotlib notebook
h.load_file('stdrun.hoc')

1.0

In [3]:
class BallAndStick:
    def __init__(self, gid):
        self._gid = gid
        self._setup_morphology()
        self._setup_biophysics()
    def _setup_morphology(self):
        self.soma = h.Section(name='soma', cell=self)
        self.dend = h.Section(name='dend', cell=self)
        self.all = [self.soma, self.dend]
        self.dend.connect(self.soma)
        self.soma.L = self.soma.diam = 12.6157
        self.dend.L = 200
        self.dend.diam = 1
    def _setup_biophysics(self):
        for sec in self.all:
            sec.Ra = 100    # Axial resistance in Ohm * cm
            sec.cm = 1      # Membrane capacitance in micro Farads / cm^2
        self.soma.insert('hh')                                          
        for seg in self.soma:
            seg.hh.gnabar = 0.12  # Sodium conductance in S/cm2
            seg.hh.gkbar = 0.036  # Potassium conductance in S/cm2
            seg.hh.gl = 0.0003    # Leak conductance in S/cm2
            seg.hh.el = -54.3     # Reversal potential in mV
        # Insert passive current in the dendrite                       # <-- NEW
        self.dend.insert('pas')                                        # <-- NEW
        for seg in self.dend:                                          # <-- NEW
            seg.pas.g = 0.001  # Passive conductance in S/cm2          # <-- NEW
            seg.pas.e = -65    # Leak reversal potential mV            # <-- NEW 
    def __repr__(self):
        return 'BallAndStick[{}]'.format(self._gid)

my_cell = BallAndStick(0)

In [4]:
from bokeh.io import output_notebook
import bokeh.plotting as bplt
from scipy.signal import butter, lfilter
import numpy as np
output_notebook()

Loading BokehJS ...

In [5]:
def butter_bandpass(lowcut, highcut, fs, order=3):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [10]:
# stim = h.MyClamp(my_cell.soma(0.5))

# print(', '.join(item for item in dir(stim) if not item.startswith('__')))

# stim.delay = 5
# stim.dur = 2000
# stim.pkamp1 = 0
# stim.pkamp2 = 3
# stim.freq1 = 2020
# stim.freq2 = 2000


soma_v = h.Vector().record(my_cell.soma(1)._ref_v)
t = h.Vector().record(h._ref_t)
#============ test vector stimulation
arrayT = np.linspace(0, 1000, 2000+1)
VecT = h.Vector(arrayT)
VecStim = h.Vector(np.sin(2*np.pi*2000*arrayT/1000)+np.sin(2*np.pi*2020*arrayT/1000))
stim = h.IClamp(my_cell.soma(0.5))
stim.dur = 400
VecStim.play(stim._ref_amp, VecT, 1)
#============



h.finitialize(-65 * mV)
h.continuerun(1000 * ms)


f = bplt.figure(x_axis_label='t (ms)', y_axis_label='v (mV)',plot_width=1000)
f.line(t, soma_v, line_width=2)
bplt.show(f)

# numpy format and filter
%matplotlib inline
import matplotlib.pyplot as plt
data = soma_v.as_numpy()
# current = y.as_numpy()
time = t.as_numpy()

fs = 1/(h.dt/1000)
filt_data = butter_bandpass_filter(data, 50, 500, fs, order=3)
# plt.figure()
# plt.plot(t,filt_data)
# # plt.plot(t,data)
# plt.show()
f = bplt.figure(x_axis_label='t (ms)', y_axis_label='v (mV)',plot_width=1000)
f.line(t, filt_data, line_width=2)
bplt.show(f)